In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.types import Date, Numeric, Text, BigInteger
from sqlalchemy.dialects.postgresql import NUMERIC
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2 import sql
import unidecode
from pathlib import Path
from typing import Optional
import logging

# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class FinancialDataExporter:
    """Gestionnaire d'export de données financières vers PostgreSQL"""
    
    def __init__(self, client_name: str, db_config: dict):
        """
        Initialise l'exporteur
        
        Args:
            client_name: Nom du client (ex: "90236_Mecahome_Sarl")
            db_config: Dict avec keys: user, password, host, port
        """
        self.client_db_name = client_name
        self.db_config = {
            'user': db_config.get('user', 'postgres'),
            'password': db_config.get('password', '566504'),
            'host': db_config.get('host', 'localhost'),
            'port': db_config.get('port', 5432)
        }
        self.engine = None
        
    def _get_connection_string(self, database: str = 'postgres') -> str:
        """Génère la chaîne de connexion PostgreSQL"""
        return (f"postgresql://{self.db_config['user']}:{self.db_config['password']}"
                f"@{self.db_config['host']}:{self.db_config['port']}/{database}")
    
    def create_database(self) -> bool:
        """Crée la base de données client si elle n'existe pas"""
        try:
            conn = psycopg2.connect(
                dbname='postgres',
                user=self.db_config['user'],
                password=self.db_config['password'],
                host=self.db_config['host']
            )
            conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
            
            with conn.cursor() as cur:
                # Vérifier si la base existe
                cur.execute(
                    "SELECT 1 FROM pg_database WHERE datname = %s",
                    (self.client_db_name,)
                )
                
                if not cur.fetchone():
                    # Créer la base avec un nom sécurisé
                    cur.execute(
                        sql.SQL("CREATE DATABASE {}").format(
                            sql.Identifier(self.client_db_name)
                        )
                    )
                    logger.info(f"Base de données '{self.client_db_name}' créée")
                else:
                    logger.info(f"Base de données '{self.client_db_name}' existe déjà")
            
            conn.close()
            
            # Créer l'engine SQLAlchemy
            self.engine = create_engine(
                self._get_connection_string(self.client_db_name)
            )
            return True
            
        except Exception as e:
            logger.error(f"Erreur création base de données: {e}")
            return False
    
    @staticmethod
    def normalize_column_name(col: str) -> str:
        """
        Normalise les noms de colonnes:
        - Supprime les accents
        - Remplace espaces et tirets par underscores
        - Convertit en minuscules
        """
        col = str(col)
        col = unidecode.unidecode(col)
        col = col.replace(" ", "_").replace("-", "_")
        return col.lower()
    
    def import_plan_comptable(self, filepath: str) -> bool:
        """Importe le plan comptable"""
        try:
            if not Path(filepath).exists():
                logger.error(f"Fichier introuvable: {filepath}")
                return False
            
            plan_df = pd.read_excel(filepath)
            plan_df.columns = [self.normalize_column_name(col) for col in plan_df.columns]
            
            plan_df.to_sql(
                "plan_comptable",
                self.engine,
                if_exists="replace",
                index=False
            )
            
            logger.info(f"Plan comptable importé: {len(plan_df)} comptes")
            return True
            
        except Exception as e:
            logger.error(f"Erreur import plan comptable: {e}")
            return False
    
    def import_grand_livre(self, filepath: str) -> Optional[pd.DataFrame]:
        """
        Importe et fusionne toutes les feuilles du grand livre
        
        Returns:
            DataFrame fusionné ou None en cas d'erreur
        """
        try:
            if not Path(filepath).exists():
                logger.error(f"Fichier introuvable: {filepath}")
                return None
            
            # Lire toutes les feuilles
            excel_file = pd.ExcelFile(filepath)
            logger.info(f"Lecture de {len(excel_file.sheet_names)} feuilles")
            
            # Fusionner
            dfs = []
            for sheet in excel_file.sheet_names:
                df = pd.read_excel(filepath, sheet_name=sheet)
                dfs.append(df)
            
            full_df = pd.concat(dfs, ignore_index=True)
            logger.info(f"Total lignes fusionnées: {len(full_df):,}")
            
            # Normaliser les colonnes
            full_df.columns = [self.normalize_column_name(col) for col in full_df.columns]
            
            # Conversion Date
            if 'date' in full_df.columns:
                full_df['date'] = pd.to_datetime(
                    full_df['date'],
                    format="%d.%m.%Y",
                    errors='coerce'
                ).dt.date
            
            # Conversion numériques avec arrondi à 2 décimales
            numeric_cols = ['debit', 'credit', 'solde', 'contre_ecr', 'document']
            for col in numeric_cols:
                if col in full_df.columns:
                    full_df[col] = pd.to_numeric(
                        full_df[col].astype(str).str.replace(",", "."),
                        errors='coerce'
                    ).round(2)
            
            # Conversion compte
            if 'compte' in full_df.columns:
                full_df['compte'] = full_df['compte'].astype('Int64')
            
            return full_df
            
        except Exception as e:
            logger.error(f"Erreur import grand livre: {e}")
            return None
    
    def export_grand_livre(self, df: pd.DataFrame) -> bool:
        """Exporte le grand livre vers PostgreSQL"""
        try:
            dtype_mapping = {
                "date": Date,
                "texte": Text,
                "compte": BigInteger,
                "contre_ecr": NUMERIC(12, 2),
                "code": Text,
                "origine": Text,
                "document": NUMERIC(12, 2),
                "debit": NUMERIC(12, 2),
                "credit": NUMERIC(12, 2),
                "solde": NUMERIC(12, 2)
            }
            
            # Filtrer seulement les colonnes existantes
            dtype_filtered = {
                k: v for k, v in dtype_mapping.items() 
                if k in df.columns
            }
            
            df.to_sql(
                "grand_livre",
                self.engine,
                if_exists="replace",
                index=False,
                dtype=dtype_filtered
            )
            
            logger.info(f"Grand livre exporté: {len(df):,} lignes")
            return True
            
        except Exception as e:
            logger.error(f"Erreur export grand livre: {e}")
            return False
    
    def import_financial_statements(self, filepath: str) -> bool:
        """Importe le bilan et compte de résultat"""
        try:
            if not Path(filepath).exists():
                logger.error(f"Fichier introuvable: {filepath}")
                return False
            
            # Import bilan
            bilan_df = pd.read_excel(filepath, sheet_name="Balance Sheet")
            bilan_df.columns = [self.normalize_column_name(col) for col in bilan_df.columns]
            
            # Arrondir toutes les colonnes numériques à 2 décimales
            for col in bilan_df.select_dtypes(include=['float64', 'float32']).columns:
                bilan_df[col] = bilan_df[col].round(2)
            
            bilan_df.to_sql(
                "bilan",
                self.engine,
                if_exists="replace",
                index=False
            )
            logger.info(f"Bilan importé: {len(bilan_df)} lignes")
            
            # Import compte de résultat
            resultat_df = pd.read_excel(filepath, sheet_name="Income Statement")
            resultat_df.columns = [self.normalize_column_name(col) for col in resultat_df.columns]
            
            # Arrondir toutes les colonnes numériques à 2 décimales
            for col in resultat_df.select_dtypes(include=['float64', 'float32']).columns:
                resultat_df[col] = resultat_df[col].round(2)
            
            resultat_df.to_sql(
                "compte_de_resultat",
                self.engine,
                if_exists="replace",
                index=False
            )
            logger.info(f"Compte de résultat importé: {len(resultat_df)} lignes")
            
            return True
            
        except Exception as e:
            logger.error(f"Erreur import états financiers: {e}")
            return False
    
    def validate_data(self, df: pd.DataFrame) -> dict:
        """
        Valide l'équilibre débit/crédit
        
        Returns:
            Dict avec statistiques de validation
        """
        stats = {
            'total_lignes': len(df),
            'total_debit': 0,
            'total_credit': 0,
            'ecart': 0,
            'equilibre': False
        }
        
        if 'debit' in df.columns and 'credit' in df.columns:
            stats['total_debit'] = df['debit'].sum()
            stats['total_credit'] = df['credit'].sum()
            stats['ecart'] = stats['total_debit'] - stats['total_credit']
            stats['equilibre'] = abs(stats['ecart']) < 0.01
        
        return stats
    
    def run_full_import(
        self,
        plan_comptable_file: str = "Plan_Comptable.xlsx",
        comptes_file: str = "Comptes_Cleans.xlsx",
        financial_statements_file: str = "Financial_Statements.xlsx"
    ) -> bool:
        """
        Exécute l'import complet
        
        Returns:
            True si succès, False sinon
        """
        logger.info("=" * 60)
        logger.info(f"DÉBUT IMPORT - Client: {self.client_db_name}")
        logger.info("=" * 60)
        
        # 1. Créer la base
        if not self.create_database():
            return False
        
        # 2. Importer plan comptable
        if not self.import_plan_comptable(plan_comptable_file):
            logger.warning("Import plan comptable échoué (non bloquant)")
        
        # 3. Importer grand livre
        grand_livre_df = self.import_grand_livre(comptes_file)
        if grand_livre_df is None:
            return False
        
        # 4. Exporter grand livre
        if not self.export_grand_livre(grand_livre_df):
            return False
        
        # 5. Valider les données
        stats = self.validate_data(grand_livre_df)
        logger.info("=" * 60)
        logger.info("VALIDATION DES DONNÉES")
        logger.info(f"→ Total lignes: {stats['total_lignes']:,}")
        logger.info(f"→ Total débit: {stats['total_debit']:,.2f} CHF")
        logger.info(f"→ Total crédit: {stats['total_credit']:,.2f} CHF")
        logger.info(f"→ Écart: {stats['ecart']:,.2f} CHF")
        logger.info(f"→ Équilibre: {'✓ PARFAIT' if stats['equilibre'] else '✗ À VÉRIFIER'}")
        logger.info("=" * 60)
        
        # 6. Importer états financiers
        if not self.import_financial_statements(financial_statements_file):
            logger.warning("Import états financiers échoué (non bloquant)")
        
        logger.info("=" * 60)
        logger.info("IMPORT TERMINÉ AVEC SUCCÈS ✓")
        logger.info("=" * 60)
        
        return True


# =============================================
# UTILISATION
# =============================================
if __name__ == "__main__":
    # Configuration
    exporter = FinancialDataExporter(
        client_name="90236_Mecahome_Sarl",
        db_config={
            'user': 'postgres',
            'password': '566504',
            'host': 'localhost',
            'port': 5432
        }
    )
    
    # Lancer l'import complet
    success = exporter.run_full_import(
        plan_comptable_file="Plan_Comptable.xlsx",
        comptes_file="Comptes_Cleans.xlsx",
        financial_statements_file="Financial_Statements.xlsx"
    )
    
    if success:
        print("\n✓ Toutes les données ont été importées avec succès!")
    else:
        print("\n✗ Des erreurs se sont produites lors de l'import")

2025-11-20 17:04:59,420 - INFO - ============================================================
2025-11-20 17:04:59,422 - INFO - DÉBUT IMPORT - Client: 90236_Mecahome_Sarl
2025-11-20 17:04:59,422 - INFO - ============================================================
2025-11-20 17:04:59,596 - INFO - Base de données '90236_Mecahome_Sarl' existe déjà
2025-11-20 17:05:00,023 - INFO - Plan comptable importé: 108 comptes
2025-11-20 17:05:00,270 - INFO - Lecture de 108 feuilles
2025-11-20 17:05:26,157 - INFO - Total lignes fusionnées: 21,339
2025-11-20 17:05:27,243 - INFO - Grand livre exporté: 21,339 lignes
2025-11-20 17:05:27,245 - INFO - ============================================================
2025-11-20 17:05:27,246 - INFO - VALIDATION DES DONNÉES
2025-11-20 17:05:27,247 - INFO - → Total lignes: 21,339
2025-11-20 17:05:27,248 - INFO - → Total débit: 6,827,708.80 CHF
2025-11-20 17:05:27,248 - INFO - → Total crédit: 6,951,432.96 CHF
2025-11-20 17:05:27,249 - INFO - → Écart: -123,724.16 CHF


✓ Toutes les données ont été importées avec succès!
